In [9]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from scipy.optimize import minimize

# 1. Historical Data
X_train = np.array([
    [0.5, 0.5],             # Week 1
    [0.15, 0.15],           # Week 2
    [1.718776, 0.354284]    # Week 3
])

y_train = np.array([
    2.6752879910742468e-09, # Week 1 (Best)
    7.257431239719379e-105, # Week 2
    0.0                     # Week 3
])

# 2. Log-Transformation (Crucial for "Needle in Haystack" functions)
# We add a tiny epsilon to avoid log(0)
y_safe = y_train.copy()
y_safe[y_safe <= 0] = 1e-300
y_log = np.log(y_safe)

# 3. Model Training (Ridge Regression to reduce overfitting)
model = make_pipeline(PolynomialFeatures(degree=2), Ridge(alpha=0.1))
model.fit(X_train, y_log)

# 4. Define Prediction Function
def objective_function(x):
    return -model.predict(x.reshape(1, -1))[0]

# 5. Trust Region Optimization
# We assume the true peak is near Week 1.
# We restrict the search to a small box +/- 0.1 around [0.5, 0.5].
# This prevents the model from running away to [0, 2].
center = np.array([0.5, 0.5])
trust_radius = 0.1
bounds = [
    (center[0] - trust_radius, center[0] + trust_radius), 
    (center[1] - trust_radius, center[1] + trust_radius)
]

result = minimize(objective_function, center, bounds=bounds, method='L-BFGS-B')

print("--- Function 1 Prediction for Week 4 ---")
print(f"Optimal Input found: {result.x}")



--- Function 1 Prediction for Week 4 ---
Optimal Input found: [0.4 0.6]
